# Baseline for an Open-Source Deep Research
This workshop will involve reasoning models and some tool usage, although no conditional execution controlled by the agent.

# Import Libraries
We'll be using Transformers and Unsloth for handling large language models,
Milvus for vector databases, LangChain and regex for text processing, and Wikipedia for external data, and json, tqdm, and typing for  general Python utilities

In [ ]:
#!pip install pymilvus unsloth wikipediaapi langchain
!pip3 uninstall pymilvus
!pip3 install pymilvus
import pymilvus
!pip3 uninstall unsloth
!pip3 install unsloth
!pip3 uninstall wandb
!pip3 install wandbn
from unsloth import FastLanguageModel
import transformers
from transformers import TextStreamer
import regex as re
!pip3 install wikipedia-api
import wikipediaapi
import json
from tqdm import tqdm
!pip3 install langchain_huggingface langchain_core langchain_text_splitters langchain_community langchain_milvus
from langchain_huggingface import HuggingFacePipeline as Pipeline
from langchain_huggingface import ChatHuggingFace as Chat
from langchain_core.messages import HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_milvus import Milvus, Zilliz
from typing import Any, Set
import json
!pip3 install json-repair
from json_repair import repair_json
from typing import Any, Set
!pip uninstall bitsandbytes
!pip install -i https://pypi.org/simple/ bitsandbytes
import bitsandbytes

Found existing installation: pymilvus 2.5.6
Uninstalling pymilvus-2.5.6:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/pymilvus-2.5.6.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pymilvus/*
Proceed (Y/n)? Y
  Successfully uninstalled pymilvus-2.5.6
  Using cached pymilvus-2.5.6-py3-none-any.whl.metadata (5.7 kB)
Using cached pymilvus-2.5.6-py3-none-any.whl (223 kB)


Found existing installation: unsloth 2025.3.19
Uninstalling unsloth-2025.3.19:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/unsloth-2025.3.19.dist-info/*
    /usr/local/lib/python3.11/dist-packages/unsloth/*
Proceed (Y/n)? Y
  Successfully uninstalled unsloth-2025.3.19
  Using cached unsloth-2025.3.19-py3-none-any.whl.metadata (46 kB)
Using cached unsloth-2025.3.19-py3-none-any.whl (192 kB)


ERROR: Could not find a version that satisfies the requirement wandbn (from versions: none)
ERROR: No matching distribution found for wandbn
Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/bitsandbytes-0.45.5.dist-info/*
    /usr/local/lib/python3.11/dist-packages/bitsandbytes/*
Proceed (Y/n)? Y
  Successfully uninstalled bitsandbytes-0.45.5
Looking in indexes: https://pypi.org/simple/
  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl (76.1 MB)


## Load reasoning and embedding models

In [ ]:
model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"
max_seq_length = 4048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

embeddings = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Helper methods
Some useful methods for chatting with the reasoning model, extracting JSON from the output, and converting the JSON into a list of strings.

In [ ]:
default_system_prompt = "You are a helpful assistant who answers question truthfully to the best of your knowledge."

def ask_model(prompt, system_prompt=default_system_prompt):
    chat = [
    {
        "role": "system",
        "content": system_prompt,
    },

    {

        "role": "user",
        "content": f"{prompt}",
    },
]

    formatted_prompt = tokenizer.apply_chat_template(
        chat, tokenize=False, add_generation_prompt=True, return_tensors="pt"
    )
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    streamer = TextStreamer(
        tokenizer, skip_prompt=True, skip_special_tokens=True
    )

    results = model.generate(**inputs, streamer=streamer, max_new_tokens=4048)
    return tokenizer.decode(token_ids=results.cpu().numpy().tolist()[0])

json_re = re.compile(r"```json\n(?s:.)*\n```")

def extract_json(response):
    # TODO: Error handling
    try:
        match = json_re.search(response)
        json_results = '\n'.join(match.group().splitlines()[1:-1])
    except:
        return {}
    return json.loads(repair_json(json_results))

def leaves(struct: Any) -> Set[Any]:
    """Return a set of leaf values found in nested dicts and lists excluding None values."""
    # Ref: https://stackoverflow.com/a/59832594/
    values = set()

    def add_leaves(struct_: Any) -> None:
        if isinstance(struct_, dict):
            for sub_struct in struct_.values():
                add_leaves(sub_struct)
        elif isinstance(struct_, list):
            for sub_struct in struct_:
                add_leaves(sub_struct)
        elif struct_ is not None:
            values.add(struct_)

    add_leaves(struct)
    return values

## Define / refine question
Break down question into sub-questions, sub-sub-questions, and so on. Convert the main question into a report title.

In [ ]:
query = "How has The Simpsons changed over time?"
page_title = "The Simpsons"

In [ ]:
prompt = f"""What is the topic of the following question? Respond in JSON format.

Question: {query}"""

response = ask_model(prompt)

Okay, so I need to figure out the topic of the question "How has The Simpsons changed over time?" Hmm, let's break it down. The question is asking about changes in The Simpsons. I know The Simpsons is a popular TV show, so it's probably about how the show evolved from its beginning to now.

First, I should consider what The Simpsons is known for. It's a long-running animated series that parodies real life and pop culture. Over the years, the show has been on for decades, so it's definitely changed a lot. I remember the early seasons being more satirical and sometimes controversial, but over time, maybe they became more family-friendly or something.

I think the topic is about the evolution of the show in terms of its content, style, or themes. It's not just about the characters changing, but how the show as a whole has developed. Maybe the writing style, the humor, the portrayal of the characters, or how it reflects society's changes.

I should also think about specific examples. Like,

In [ ]:
prompt = f"""Break down the following question into intermediate sub-questions to approach answering it. Provide a list of intermediate sub-questions and respond with JSON format. If you cannot produce sub-question then say so. Do not directly answer the following question and only return the sub-questions in JSON format. Your answer must contain JSON.

Question: {query}"""

response = ask_model(prompt)

Okay, so I need to break down the question "How has The Simpsons changed over time?" into intermediate sub-questions. Hmm, where do I start? Well, I think the first thing I should consider is understanding the overall scope of the question. It's about The Simpsons, a long-running show, and how it has evolved. So, I should probably think about different aspects of the show that have changed.

Maybe the initial version of The Simpsons was different from the current one. So, one sub-question could be about the evolution of the main characters. I know that characters like Homer, Marge, Bart, and Lisa have changed over the years, so that makes sense. Then, the humor must have evolved too. The original show was funny but maybe in a different way than it is now. So another sub-question could be about the changes in humor.

The format of the show is another area. I've heard that The Simpsons started as a traditional animation but then moved to a more modern style. That's a good point for a sub

In [ ]:
sub_questions = list(leaves(extract_json(response)))

In [ ]:
breakdown = {}

# Hardcode topic for now from output above
topic = "The evolution of The Simpsons as a show over time, covering changes in content, humor, character development, animation, and its role in society."

# Break sub-questions into sub-sub-questions
for q in sub_questions:
    prompt = f"""You are researching the follow topic. Break down the following question into intermediate sub-questions to approach answering it. Provide a list of intermediate sub-questions and respond with JSON format. If you cannot produce sub-questions then say so. Do not directly answer the following question and only return the sub-questions in JSON format. Your answer must contain JSON.

    Topic: {topic}

    Question: {q}"""

    response = ask_model(prompt)
    sub_sub_questions = list(leaves(extract_json(response)))
    breakdown[q] = sub_sub_questions


Okay, so I need to figure out how the content and themes of The Simpsons have evolved over time. I'm not super familiar with the show, but I know it's been on for a while, so I guess it must have changed a lot. Let me break this down.

First, I should probably consider the initial seasons. I remember hearing that The Simpsons started as a show about a regular family, right? Homer is the dad, Marge is the mom, Bart is the son, and Lisa is the daughter. They live in Springfield. The first few seasons were probably more lighthearted, maybe focusing on family life and everyday situations. The humor was probably more slapstick or situational, not as complex as it is now.

Then, as the show went on, I think the content got more edgy. I've heard that the later seasons tackled more mature themes. Maybe they started satirizing other shows or societal issues. For example, they parodied other genres like crime dramas, musicals, and even politics. So the humor became more satirical and sometimes c

In [ ]:
sub_questions = list(leaves(extract_json(response)))

In [ ]:
breakdown

{'How has the content and themes of The Simpsons evolved?': ['How did the content and themes of The Simpsons evolve over time?',
  'How did the animation style of The Simpsons change over the years?',
  'What is the role of The Simpsons in the evolution of society and pop culture?',
  "How did character development change throughout the show's run?",
  'What were the initial content and themes of The Simpsons in its early seasons?'],
 'What are the differences in the animation style of The Simpsons over its run?': [],
 'What is the legacy and overall impact of The Simpsons?': [],
 'How have the voice actors changed over time for The Simpsons?': ['What are the reasons for changes in the voice cast?',
  'Who were the replacements for key characters and what impact did they have?',
  'What was the initial voice cast of The Simpsons?',
  'How have the voice actors changed over time for The Simpsons?',
  'What is the role of the voice actors in the evolution of The Simpsons?'],
 'How has th

## Search

### Build vector database on Wikipedia article

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia(user_agent='MilvusDeepResearchBot (<insert your email>)', language='en')
page_py = wiki_wiki.page(page_title)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.create_documents([page_py.text])

In [ ]:
vectorstore = Milvus.from_documents(  # or Zilliz.from_documents
    documents=docs,
    embedding=embeddings,
    connection_args={
        "uri": "./milvus_demo.db",
    },
    drop_old=True,  # Drop the old Milvus collection if it exists
    index_params={
        "metric_type": "COSINE",
        "index_type": "FLAT",  # <= NOTE: Currently a bug where langchain_milvus defaults to "HNSW" index, which doesn't work with Milvus Lite
        "params": {},
    },
)

### Break down question in sub-questions, and so on.

In [ ]:
q = 'How has the cast changed over time?'

def question_to_header(question, topic=None):
    if topic is None:
        prompt = f"""Rewrite the following question as a header title. Be concise. Respond in JSON format with an escaped code block.

Question: {q}"""
    else:
        prompt = f"""Rewrite the following question with given context as a header title. Be concise. Respond in JSON format with an escaped code block.

Context: {topic}
Question: {q}"""

    response = ask_model(prompt)
    return list(leaves(extract_json(response)))[0]

In [ ]:
header = question_to_header(q, topic)

Okay, so I need to figure out how to rewrite the question "How has the cast changed over time?" into a header title based on the given context. The context is about the evolution of The Simpsons as a show, covering changes in content, humor, character development, animation, and its role in society. 

First, I should understand what the question is asking. It's about the changes in the cast over time. So, the header needs to reflect that it's discussing the evolution or changes in the cast. 

Looking at the context, the show has changed in various aspects: content, humor, characters, animation, and societal impact. So, the header should capture that transformation of the cast over the years. 

Possible words that come to mind: evolution, changes, cast, over time. Maybe something like "Evolution of the Cast Over Time" or "Changes in the Cast Over Years." But the example given uses "Evolution of the Cast Over Time," which is concise and fits well. 

I think "The Evolution of the Cast Ove

In [ ]:
from langchain_core.messages import HumanMessage

import transformers
from langchain_huggingface import HuggingFacePipeline as Pipeline
from langchain_huggingface import ChatHuggingFace as Chat

FastLanguageModel.for_inference(model)

hf_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    # device="cuda",
    # repetition_penalty=1.15,
    return_full_text=False,
    max_new_tokens=4048,
    # output_scores=True,
    # use_cache=False,
    # truncation=True
)

llm = Pipeline(pipeline=hf_pipeline)
chat = Chat(llm=llm)

Device set to use cuda:0


tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Analyze
Answer (sub-)sub-questions

In [ ]:
# DEBUG: Without providing context
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt template for generating AI responses
PROMPT_TEMPLATE = """
You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in $question$ tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Answer in a single short paragraph.
$context$
{context}
$/context$

$question$
{question}
$/question$
"""

# Create a PromptTemplate instance with the defined template and input variables
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["question"]
)
# Convert the vector store to a retriever
retriever = vectorstore.as_retriever()

# Define a function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG (Retrieval-Augmented Generation) chain for AI response generation
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# rag_chain.get_graph().print_ascii()

In [ ]:
# Prompt the RAG for each question
answers = {}
total = len(leaves(breakdown)) + 4

pbar = tqdm(total=total)
for k, v in breakdown.items():
    if v == []:
        print(k)
        answers[k] = rag_chain.invoke(k).split('</think>')[-1].strip()
        pbar.update(1)
    else:
        for q in v:
            print(q)
            answers[q] = rag_chain.invoke(q).split('</think>')[-1].strip()
            pbar.update(1)

  0%|          | 0/43 [00:00<?, ?it/s]

How did the content and themes of The Simpsons evolve over time?


  2%|▏         | 1/43 [00:57<40:10, 57.40s/it]

How did the animation style of The Simpsons change over the years?


  5%|▍         | 2/43 [01:44<35:10, 51.48s/it]

...

Personal growth


 47%|████▋     | 20/43 [12:12<14:07, 36.83s/it]

Personal struggles


 49%|████▉     | 21/43 [12:59<14:34, 39.77s/it]

How has Mr. Burns's character evolved over the series?


In [ ]:
answers

In [ ]:
import pickle
with open('answers.pkl', 'wb') as f:
    pickle.dump(answers, f)

## Synthesize

In [ ]:
report = [f'# {topic}\n\n']
for k, v in breakdown.items():
    report.append(f'## {k}\n')
    if v == []:
        report.append(answers[k] + '\n\n')

    else:
        for q in v:
            report.append(f'### {q}\n')
            report.append(answers[q] + '\n\n')

In [ ]:
md = ''.join(report)
with open('report.md', 'w') as f:
    print(md, file=f)

In [ ]:
md